In [13]:
import pandas as pd # it is used for the time series management dataset.
import numpy as np # it is used to manage collections of values in the pandas dataframe.
import json 
import datetime as dt
import warnings
import math
import matplotlib.pyplot as plt
import pickle
from keras.models import load_model

In [14]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.callbacks import ModelCheckpoint
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError
from keras import metrics
from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [15]:
from keras.callbacks import EarlyStopping
stop = EarlyStopping(monitor='root_mean_squared_error', patience=30, mode='min')

In [16]:
#Load dataset
df_grouped = pd.read_pickle('../../monthly/grouped_all.pkl')

In [ ]:
df_grouped.head()

In [ ]:
# filter points from original dataset to geojson coordinates related to river positions.
coords = json.load(open('../app/static/data/points_new_fill.geojson', 'r'))
lats = []
longs = []
for coord in coords['features'][:]:
    sample = coord['geometry']['coordinates']
    lats.append(sample[1])
    longs.append(sample[0])


In [ ]:
datasource = json.load(open('../samples/source_mounth.json', 'r'))
sources_rivers = []
names = []
mounths_rivers = []
italy_sources_mounths = pd.DataFrame()
for key in datasource.keys():
    sources_rivers.append(datasource[key]['source'])
    mounths_rivers.append(datasource[key]['mounth'])
    names.append(key)

italy_sources_mounths['names'] = names
italy_sources_mounths['mounth'] = mounths_rivers
italy_sources_mounths['source'] = sources_rivers
italy_sources_mounths['number_value'] = [i for i in range(0, len(italy_sources_mounths))]
italy_sources_mounths.head()

In [ ]:
#dictionaries with the dataframe for each river
sources = {}
mounths = {}
names = list(italy_sources_mounths.names)
for i in range(len(italy_sources_mounths)):
    name = names[i]
    sources[name] = df[df['coords'] == tuple(italy_sources_mounths['source'][i])]
    mounths[name] = df[df['coords'] == tuple(italy_sources_mounths['mounth'][i])]

In [ ]:
for name in sources:
    #sources[name] = sources[name].drop(columns=['coords'], inplace=False)
    sources[name] = sources[name].set_index('time')
    #mounths[name] = mounths[name].drop(columns=['coords'], inplace=False)
    mounths[name] = mounths[name].set_index('time')

In [22]:
#conversion from a single column dataframe to two arrays, one with the observations and one with the label to predict
def df_to_supervised_discharge(df, window_size=12):
    df_as_np = df.to_numpy()
    X = []
    y = []
    for i in range(len(df_as_np)-window_size):
        row = [r for r in df_as_np[i:i+window_size]]
        X.append(row)
        label = df_as_np[i+window_size][0]
        y.append(label)
    return np.array(X), np.array(y)

In [23]:
#conversion from a single column dataframe to two arrays, one with the observations and one with the label to predict
def df_to_supervised_temperature(df, window_size=12):
    df_as_np = df.to_numpy()
    X = []
    y = []
    for i in range(len(df_as_np)-window_size):
        row = [r for r in df_as_np[i:i+window_size]]
        X.append(row)
        label = df_as_np[i+window_size][0]
        y.append(label)
    return np.array(X), np.array(y)

In [ ]:
#add column with seconds to do the sin cos trick
for name in sources:
    sources[name]['secs'] = sources[name].index.map(pd.Timestamp.timestamp)
    mounths[name]['secs'] = mounths[name].index.map(pd.Timestamp.timestamp)

In [ ]:

day = 60*60*24
year = 365.2425*day
for name in sources:
    sources[name]['year_sin'] = np.sin(sources[name]['secs']*(2*np.pi/year))
    mounths[name]['year_sin'] = np.sin(mounths[name]['secs']*(2*np.pi/year))

    sources[name]['year_cos'] = np.cos(sources[name]['secs']*(2*np.pi/year))
    mounths[name]['year_cos'] = np.cos(mounths[name]['secs']*(2*np.pi/year))

    sources[name] = sources[name].drop(columns=['secs'])
    mounths[name] = mounths[name].drop(columns=['secs'])

In [ ]:
sources['Tevere '].drop(columns=['prec', 'discharge'])

In [ ]:
for name in sources:
    src = sources[name].drop(columns=['prec', 'discharge', 'coords'])
    #creating the x tensor and the labels
    X, y = df_to_supervised_temperature(src)

    #splitting
    n = len(X)
    X_train, y_train = X[0:int(n*0.7)], y[0:int(n*0.7)]
    X_val, y_val = X[int(n*0.7):int(n*0.9)], y[int(n*0.7):int(n*0.9)]
    X_test, y_test = X[int(n*0.9):], y[int(n*0.9):]

    model_tmp_src = Sequential()
    model_tmp_src.add(LSTM(32, input_shape=(X_train.shape[1], X_train.shape[2])))
    model_tmp_src.add(Dense(12))
    model_tmp_src.add(Dense(1))

    #model2.summary()
    model_tmp_src.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.1), metrics=[RootMeanSquaredError()])
    model_tmp_src.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=30, verbose=0, callbacks=[stop])

    test_predictions = model_tmp_src.predict(X_test).flatten()
    test_results = pd.DataFrame(data={'Preds': test_predictions, 'Actuals':y_test})
    mse = mean_squared_error(test_predictions, y_test)
    mae = mean_absolute_error(test_predictions, y_test)
    print('Test on sources for '+name)
    print('MSE: %.3f' % mse)
    print('MAE: %.3f' % mae)

    plt.plot(test_results)
    plt.show()

In [ ]:
for name in sources:
    src = mounths[name].drop(columns=['prec', 'discharge', 'coords'])
    #creating the x tensor and the labels
    X, y = df_to_supervised_temperature(src)

    #splitting
    n = len(X)
    X_train, y_train = X[0:int(n*0.7)], y[0:int(n*0.7)]
    X_val, y_val = X[int(n*0.7):int(n*0.9)], y[int(n*0.7):int(n*0.9)]
    X_test, y_test = X[int(n*0.9):], y[int(n*0.9):]

    model_tmp_mth = Sequential()
    model_tmp_mth.add(LSTM(32, input_shape=(X_train.shape[1], X_train.shape[2])))
    model_tmp_mth.add(Dense(12))
    model_tmp_mth.add(Dense(1))

    #model2.summary()
    model_tmp_mth.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.1), metrics=[RootMeanSquaredError()])
    model_tmp_mth.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=30, verbose=0)
    test_predictions = model_tmp_mth.predict(X_test).flatten()
    test_results = pd.DataFrame(data={'Preds': test_predictions, 'Actuals':y_test})
    mse = mean_squared_error(test_predictions, y_test)
    mae = mean_absolute_error(test_predictions, y_test)
    print('Test on mouths for '+name)
    print('MSE: %.3f' % mse)
    print('MAE: %.3f' % mae)
    plt.plot(test_predictions, label='predictions')
    plt.plot(y_test, label='actual')
    plt.legend(loc="upper right")
    plt.show()

In [ ]:
for name in sources:
    src = sources[name].drop(columns=['prec'])
    #creating the x tensor and the labels
    X, y = df_to_supervised_discharge(src)

    #splitting
    n = len(X)
    X_train, y_train = X[0:int(n*0.7)], y[0:int(n*0.7)]
    X_val, y_val = X[int(n*0.7):int(n*0.9)], y[int(n*0.7):int(n*0.9)]
    X_test, y_test = X[int(n*0.9):], y[int(n*0.9):]
    #normalizing
    X_mean = X_train.mean()
    X_std = X_train.std()
    X_train_sc = (X_train - X_mean) / X_std
    X_val_sc = (X_val - X_mean) / X_std
    X_test_sc = (X_test - X_mean) / X_std


    y_mean = y_train.mean()
    y_std = y_train.std()
    y_train_sc = (y_train - X_mean) / X_std
    y_val_sc = (y_val - X_mean) / X_std
    y_test_sc = (y_test - X_mean) / X_std
    
    model_dis_src = Sequential()
    model_dis_src.add(LSTM(32, input_shape=(X_train_sc.shape[1], X_train_sc.shape[2])))
    model_dis_src.add(Dense(12))
    model_dis_src.add(Dense(1))

    #model2.summary()
    model_dis_src.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.1), metrics=[RootMeanSquaredError()])
    model_dis_src.fit(X_train_sc, y_train_sc, validation_data=(X_val_sc, y_val_sc), epochs=30, verbose=0)

    test_predictions = model_dis_src.predict(X_test_sc).flatten()
    test_predictions = (test_predictions*X_std)+X_mean
    test_results = pd.DataFrame(data={'Preds': test_predictions, 'Actuals':y_test})
    mse = mean_squared_error(test_predictions, y_test)
    mae = mean_absolute_error(test_predictions, y_test)
    print('Test on mouths for '+name)
    print('MSE: %.3f' % mse)
    print('MAE: %.3f' % mae)
    plt.plot(test_results)
    plt.show()

In [ ]:
for name in mounths:
    src = mounths[name].drop(columns=['prec'])
    #creating the x tensor and the labels
    X, y = df_to_supervised_discharge(src)

    #splitting
    n = len(X)
    X_train, y_train = X[0:int(n*0.7)], y[0:int(n*0.7)]
    X_val, y_val = X[int(n*0.7):int(n*0.9)], y[int(n*0.7):int(n*0.9)]
    X_test, y_test = X[int(n*0.9):], y[int(n*0.9):]
    #normalizing
    X_mean = X_train.mean()
    X_std = X_train.std()
    X_train_sc = (X_train - X_mean) / X_std
    X_val_sc = (X_val - X_mean) / X_std
    X_test_sc = (X_test - X_mean) / X_std


    y_mean = y_train.mean()
    y_std = y_train.std()
    y_train_sc = (y_train - y_mean) / y_std
    y_val_sc = (y_val - y_mean) / y_std
    y_test_sc = (y_test - y_mean) / y_std
    
    model_dis_mth = Sequential()
    model_dis_mth.add(LSTM(32, input_shape=(X_train_sc.shape[1], X_train_sc.shape[2])))
    model_dis_mth.add(Dense(12))
    model_dis_mth.add(Dense(1))

    #model2.summary()
    model_dis_mth.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.1), metrics=[RootMeanSquaredError()])
    model_dis_mth.fit(X_train_sc, y_train_sc, validation_data=(X_val_sc, y_val_sc), epochs=30, verbose=0)

    test_predictions = model_dis_mth.predict(X_test_sc).flatten()
    test_predictions = (test_predictions*X_std)+X_mean
    test_results = pd.DataFrame(data={'Preds': test_predictions, 'Actuals':y_test})
    rmse = math.sqrt(mean_squared_error(test_predictions, y_test))
    print('Test on mouths for '+name)
    print('RMSE: %.3f' % rmse)
    plt.plot(test_results)
    plt.show()

In [ ]:
from keras.callbacks import EarlyStopping
stop = EarlyStopping(monitor='root_mean_squared_error', patience=30, mode='min')

### Future prediction

In [19]:
def create_future_df(df, window_size=12):
    last_time = df.index.values[-1]
    df_as_np = df.to_numpy()
    X = []
    i = len(df_as_np)-window_size
    row = [r for r in df_as_np[i:i+window_size]]
    X.append(row)
    return np.array(X), last_time

In [20]:

def append_row_tmp(X, last_time, predicted_val):
    #create the new row basically
    X_new = []
    day = 60*60*24
    year = 365.2425*day
    next_month = pd.Timestamp(last_time) + pd.DateOffset(months=1)
    sin = np.sin(next_month.timestamp()*(2*np.pi/year))
    cos = np.cos(next_month.timestamp()*(2*np.pi/year))
    row = [predicted_val, sin, cos]
    for i in range(1, len(X[0])):
        X_new.append(X[0][i])
    X_new.append(row)
    return np.array([X_new]), next_month

In [21]:
def append_row_dis(X, last_time, predicted_val_tmp, predicted_val_dis):
    #create the new row basically
    X_new = []
    day = 60*60*24
    year = 365.2425*day
    next_month = pd.Timestamp(last_time) + pd.DateOffset(months=1)
    sin = np.sin(next_month.timestamp()*(2*np.pi/year))
    cos = np.cos(next_month.timestamp()*(2*np.pi/year))
    row = [predicted_val_dis, predicted_val_tmp, sin, cos]
    for i in range(1, len(X[0])):
        X_new.append(X[0][i])
    X_new.append(row)
    return np.array([X_new]), next_month

In [ ]:
src_tmp = sources['Ticino'].drop(columns=['prec', 'discharge', 'coords'])
src_dis = sources['Ticino'].drop(columns=['prec', 'coords'])
X_tmp, last_time = create_future_df(src_tmp)
X_dis, _ = create_future_df(src_dis)

In [ ]:
#creating the x tensor and the labels
X, y = df_to_supervised_temperature(src_tmp)

#splitting
n = len(X)
X_train, y_train = X[0:int(n*0.7)], y[0:int(n*0.7)]
X_val, y_val = X[int(n*0.7):], y[int(n*0.7):]

model_tmp_src = Sequential()
model_tmp_src.add(LSTM(32, input_shape=(X_train.shape[1], X_train.shape[2])))
model_tmp_src.add(Dense(12))
model_tmp_src.add(Dense(1))

model_tmp_src.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.1), metrics=[RootMeanSquaredError()])
model_tmp_src.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=30, verbose=0)

In [ ]:
#creating the x tensor and the labels
X, y = df_to_supervised_discharge(src_dis)

#splitting
n = len(X)
X_train, y_train = X[0:int(n*0.7)], y[0:int(n*0.7)]
X_val, y_val = X[int(n*0.7):], y[int(n*0.7):]

#normalizing
X_mean = X_train.mean()
X_std = X_train.std()
X_train_sc = (X_train - X_mean) / X_std
X_val_sc = (X_val - X_mean) / X_std


y_mean = y_train.mean()
y_std = y_train.std()
y_train_sc = (y_train - y_mean) / y_std
y_val_sc = (y_val - y_mean) / y_std

model_dis_src = Sequential()
model_dis_src.add(LSTM(32, input_shape=(X_train_sc.shape[1], X_train_sc.shape[2])))
model_dis_src.add(Dense(12))
model_dis_src.add(Dense(1, activation='softplus'))

model_dis_src.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.01), metrics=[RootMeanSquaredError()])
model_dis_src.fit(X_train_sc, y_train_sc, validation_data=(X_val_sc, y_val_sc), epochs=1000, verbose=0, callbacks=[stop])

In [ ]:
n_pred = 12
tmp_pred_ls = []
dis_pred_ls = []
time = []
for i in range(n_pred):
    tmp_pred = model_tmp_src.predict(X_tmp).flatten()[0]
    dis_pred = model_dis_src.predict(X_dis).flatten()[0]
    tmp_pred_ls.append(tmp_pred)
    dis_pred_ls.append(dis_pred)
    time.append(last_time)
    X_tmp, _ = append_row_tmp(X_tmp, last_time, tmp_pred)
    X_dis, last_time = append_row_dis(X_dis, last_time, tmp_pred, dis_pred)

data = {'date': time, 'dis':dis_pred_ls, 'temp':tmp_pred_ls}
new_df = pd.DataFrame(data=data)
print(new_df.head())

#i already have the model trained

### Generalize

In [24]:
#creating the dataframe for each point
day = 60*60*24
year = 365.2425*day

df_prediction = df_grouped
df_prediction = df_prediction.set_index('time')
df_prediction['secs'] = df_prediction.index.map(pd.Timestamp.timestamp)

df_prediction['year_sin'] = np.sin(df_prediction['secs']*(2*np.pi/year))
df_prediction['year_cos'] = np.cos(df_prediction['secs']*(2*np.pi/year))
df_prediction = df_prediction.reset_index()
df_prediction = df_prediction.set_index(['coords', 'time'])

df_prediction = df_prediction.drop(columns=['secs'])
df_prediction.head()

discharge       temp          prec  \
coords                time                                             
(40.519886, 8.314602) 2011-01-15   0.380702  12.225464  1.492938e-05   
                      2011-02-15   0.405744   9.720506  4.443793e-05   
                      2011-03-15   0.229209  10.260750  1.142119e-05   
                      2011-04-15   0.167334  14.710362  1.063244e-05   
                      2011-05-15   0.120527  16.981652  8.759512e-07   

                                  year_sin  year_cos  
coords                time                            
(40.519886, 8.314602) 2011-01-15  0.239478  0.970902  
                      2011-02-15  0.699798  0.714341  
                      2011-03-15  0.951104  0.308870  
                      2011-04-15  0.976054 -0.217529  
                      2011-05-15  0.741586 -0.670858

In [25]:
df_grouped = None

In [ ]:
df_prediction.tail()

In [ ]:
#on each coordinate i do the prediction
parent = 'E:/francesco/UNI/HDS/models'
fd = open('../app/static/data/points_new_fill.geojson', 'r')
geojson = json.load(fd)
df = pd.DataFrame(columns=['coords', 'time', 'dis', 'temp'])
n_pred = 12
for j in range(len(geojson['features'])):
    coord = (geojson['features'][j]['geometry']['coordinates'][1], geojson['features'][j]['geometry']['coordinates'][0])
    reduced_df = df_prediction.loc[coord, :]
    #creating the datasets to train the model for the temperature
    X_t, y_t = df_to_supervised_temperature(reduced_df.drop(columns=['prec', 'discharge']))

    #splitting
    n = len(X_t)
    X_train, y_train = X_t[0:int(n*0.7)], y_t[0:int(n*0.7)]
    X_val, y_val = X_t[int(n*0.7):], y_t[int(n*0.7):]
    #it trains better on unscaled data
    model_tmp = Sequential()
    model_tmp.add(LSTM(32, input_shape=(X_train.shape[1], X_train.shape[2])))
    model_tmp.add(Dense(12))
    model_tmp.add(Dense(1))

    model_tmp.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.1), metrics=[RootMeanSquaredError()])
    model_tmp.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10000, verbose=0, callbacks=[stop])
    #saving the model inside the directory
        
    #creating the datasets to train the model for the discharge
    X_d, y_d = df_to_supervised_discharge(reduced_df.drop(columns=['prec']))

    #splitting
    n = len(X_d)
    X_train, y_train = X_d[0:int(n*0.7)], y_d[0:int(n*0.7)]
    X_val, y_val = X_d[int(n*0.7):], y_d[int(n*0.7):]

    #normalizing
    X_mean = X_train.mean()
    X_std = X_train.std()
    X_train_sc = (X_train - X_mean) / X_std
    X_val_sc = (X_val - X_mean) / X_std

    y_mean = y_train.mean()
    y_std = y_train.std()
    y_train_sc = (y_train - X_mean) / X_std
    y_val_sc = (y_val - X_mean) / X_std
    model_dis = Sequential()
    model_dis.add(LSTM(32, input_shape=(X_train_sc.shape[1], X_train_sc.shape[2])))
    model_dis.add(Dense(12))
    model_dis.add(Dense(1))

    model_dis.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.01), metrics=[RootMeanSquaredError()])
    model_dis.fit(X_train_sc, y_train_sc, validation_data=(X_val_sc, y_val_sc), epochs=10000, verbose=0, callbacks=[stop])

    #creating the new dataset from the last value to predict the future
    X_tmp, last_time = create_future_df(reduced_df.drop(columns=['prec', 'discharge']))
    X_dis, _ = create_future_df(reduced_df.drop(columns=['prec']))
    X_dis_sc = (X_dis - X_mean) / X_std
    #structure to save the prediction results
    tmp_pred_ls = []
    dis_pred_ls = []
    time = []
    coords = []
    #predicting
    for i in range(n_pred):
        tmp_pred = model_tmp.predict(X_tmp).flatten()[0]
        dis_pred = model_dis.predict(X_dis_sc).flatten()[0]
        dis_pred = (dis_pred*X_std)+X_mean
        tmp_pred_ls.append(tmp_pred)
        if(dis_pred < 0):
            dis_pred_ls.append(0)
        else:
            dis_pred_ls.append(dis_pred)
        time.append(last_time)
        coords.append(coord)
        X_tmp, _ = append_row_tmp(X_tmp, last_time, tmp_pred)
        X_dis, last_time = append_row_dis(X_dis, last_time, tmp_pred, dis_pred)
        X_dis_sc = (X_dis - X_mean) / X_std

    data = {'coords': coords, 'time': time, 'dis':dis_pred_ls, 'temp':tmp_pred_ls}
    df = pd.concat([df,  pd.DataFrame(data=data)])
    del model_tmp
    del model_dis
df.to_pickle(parent+'/grouped_pred.pkl')